In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def getNewDataFromTrain():
    '''从训练数据集合中得到orderType对应的1的userid
        增加正样本数据。
    '''
    df_train_order = pd.read_csv('../data/train/orderHistory_train.csv')
    df_train = pd.read_csv('../data/train/orderFuture_train.csv')
    newID = list(set(df_train[df_train['orderType']==1]['userid']) - \
                 set(df_train_order[df_train_order['orderType']==1]['userid']))
    
    
    return newID

def getNewDataFromTest():
    '''从测试集合中得到orderType=1对应的userid，增加新数据
    
    '''
    df_test_order = pd.read_csv('../data/test/orderHistory_test.csv')
    
    return list(df_test_order[df_test_order['orderType']==1]['userid'])
    


In [3]:
# newID从训练集中得到的orderType是1的对应的userid，用来增加新数据
newIDTrain = getNewDataFromTrain()

newIDTest = getNewDataFromTest()


In [4]:
# 从所有的数据中提取原始的数据，写入文件中
df_train_order = pd.read_csv('../data/train/orderHistory_train.csv')
df_train = pd.read_csv('../data/train/orderFuture_train.csv')


In [5]:
df_train_order_1 = df_train_order[df_train_order['orderType']==1]

In [6]:
df_train_order_1.sort_values(['userid','orderType'],inplace=True)

/home/xiaoran/Tools/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [7]:
# 因为时间不同，没有必要剔除userid相同的数据，整理要用的是

df_train_order_1.drop_duplicates('userid',keep='last',inplace=True)

/home/xiaoran/Tools/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [8]:
df_train_action = pd.read_csv('../data/train/action_train.csv')


In [9]:
df_train_action_1 = df_train_action[df_train_action['userid'].isin(df_train_order_1['userid'])]

In [10]:
df_tmp = pd.merge(df_train_action_1,df_train_order_1,how='left',on='userid')

df_tmp = df_tmp[df_tmp['actionTime'] < df_tmp['orderTime']]

In [11]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94662 entries, 0 to 155655
Data columns (total 9 columns):
userid        94662 non-null int64
actionType    94662 non-null int64
actionTime    94662 non-null int64
orderid       94662 non-null int64
orderTime     94662 non-null int64
orderType     94662 non-null int64
city          94662 non-null object
country       94662 non-null object
continent     94662 non-null object
dtypes: int64(6), object(3)
memory usage: 7.2+ MB


In [12]:
df_train_order_new = df_train_order[~df_train_order.index.isin(df_train_order_1.index)]

In [13]:
# 删除没有的userid
x = df_train_order_1[~df_train_order_1['userid'].isin(df_tmp['userid'])]

df_train_order_1 = df_train_order_1[~df_train_order_1.index.isin(x.index)]

In [14]:
df_train_order_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1701 entries, 16 to 20652
Data columns (total 7 columns):
userid       1701 non-null int64
orderid      1701 non-null int64
orderTime    1701 non-null int64
orderType    1701 non-null int64
city         1701 non-null object
country      1701 non-null object
continent    1701 non-null object
dtypes: int64(4), object(3)
memory usage: 106.3+ KB


In [15]:
df_tmp[['userid','actionType','actionTime']].to_csv('../data/xtrain/action_train.csv',index=False,index_label=False)
df_train_order_1[['userid','orderType']].to_csv('../data/xtrain/orderFuture_train.csv',index=False,index_label=False)

df_train_order_new.to_csv('../data/xtrain/orderHistory_train.csv',index=False,index_label=False)